## Clothings Inventory And Sorting with CNN

**INTRODUCTION**

*Project Overview*
- This project involves building a machine learning model that uses Convolutional Neural Network (CNN) to identify different inventory products and categorise them into specific classes where summary of the total inventory can be extracted. The processes involved in setting up the model includes data collection, cleaning, importing and installation of the needed libraries, setting parameters, data preprocessing and augmentation, model definition, compilation, training, evaluation and testing.

*Problem Statement*
- Overtime sorting and taking inventory has always been a little challenging and somewhat exhausting sometimes, engaging in both physical and mental exercises which in some cases might require repetition for accuracy. A a solution, this setup with the trained model helps to identify each product according to its category and provides a summary of available product given the pictorial data of all the products.

#### Importing librabries

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

In [2]:
tf.__version__

'2.20.0'

#### Defining image dimensions

In [3]:
img_height, img_width = 64, 64

#### Defining categories

In [4]:
categories = ['Dress', 'Pants', 'Shirt', 'Shoes'] 
num_classes = len(categories)

#### Data Preprocessing and Augmentation

#### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    )

train_generator = train_datagen.flow_from_directory(
    'dataset/training_set', 
    target_size=(img_height, img_width), 
    batch_size=32, 
    class_mode='categorical', 
    subset='training')

Found 2556 images belonging to 4 classes.


#### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

validation_generator = test_datagen.flow_from_directory(
    'dataset/test_set', 
    target_size=(img_height, img_width), 
    batch_size=32, 
    class_mode='categorical', 
    subset='validation')

Found 160 images belonging to 4 classes.


#### CNN model definition

In [7]:
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_height,img_width, 3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=1))

# Convolutional layer 2
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

# Convolutional layer 3
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [8]:
print(validation_generator.samples)
print(train_generator.samples)
print(validation_generator.class_indices)
print(train_generator.class_indices)

160
2556
{'dress': 0, 'pants': 1, 'shirts': 2, 'shoes': 3}
{'dress': 0, 'pants': 1, 'shirts': 2, 'shoes': 3}


#### Model Compilation

In [9]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary

<bound method Model.summary of <Sequential name=sequential, built=True>>

#### Model Training

In [10]:
model.fit(x = train_generator, validation_data=validation_generator, epochs=30)

Epoch 1/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 39s 449ms/step - accuracy: 0.5869 - loss: 1.0310 - val_accuracy: 0.6750 - val_loss: 0.8089
Epoch 2/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 35s 438ms/step - accuracy: 0.7269 - loss: 0.7267 - val_accuracy: 0.7500 - val_loss: 0.6106
Epoch 3/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 36s 445ms/step - accuracy: 0.7778 - loss: 0.6103 - val_accuracy: 0.8188 - val_loss: 0.4807
Epoch 4/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 35s 440ms/step - accuracy: 0.8016 - loss: 0.5335 - val_accuracy: 0.8313 - val_loss: 0.4500
Epoch 5/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 36s 447ms/step - accuracy: 0.8247 - loss: 0.4785 - val_accuracy: 0.8500 - val_loss: 0.4537
Epoch 6/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 36s 453ms/step - accuracy: 0.8490 - loss: 0.4367 - val_accuracy: 0.8750 - val_loss: 0.3777
Epoch 7/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 36s 446ms/step - accuracy: 0.8549 - loss: 0.3890 - val_accuracy: 0.9000 - val_loss: 0.3306
Epoch 8/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 35s 438ms/step - accuracy: 0.8599 - loss: 0.3794 - val_accu

#### Model Evaluation

In [11]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {round(loss, 2)}, Validation Accuracy: {round(accuracy, 2)}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.9688 - loss: 0.1037
Validation loss: 0.1, Validation Accuracy: 0.97


### Making a single prediction

In [12]:
import numpy as np
from tensorflow.keras.preprocessing import image

categories = ['Dress', 'Pant', 'Shirt', 'Shoe'] 

test_image = image.load_img('test/dress (4).jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis= 0)
result = model.predict(test_image)
train_generator.class_indices

prediction = categories[np.argmax(result)]
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Dress


#### Saving model

In [13]:
model.save('models/clothing_cnn_model_v2.h5')

#### Reusing model

In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

# Define the categories
categories = ['Dress', 'Pant', 'Shirt', 'Shoe'] 

# Load the save model
model = load_model('models/clothing_cnn_model_v2.h5')

# Function for image prediction
def predict_category(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)/255.0
    
    # Predict the class probabilities
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    category = categories[predicted_class]

    # print(f"Predicted category: {category}")
    return category

In [2]:
from pathlib import Path

# Test prediction
dress = []
shoes = []
pants = []
shirts = []
unidentified = []

folder = Path("C:/Users/fortu/Desktop/Programming/Projects/Clothings Inventory Sorting App/test")

fileNames = [file.name for file in folder.iterdir() if file.is_file()]

for i in fileNames:
    file_path = f'{folder}/{i}'
    prediction = predict_category(file_path)
    if prediction == 'Dress':
        dress.append(i)
    elif prediction == 'Shoe':
        shoes.append(i)
    elif prediction == 'Pant':
        pants.append(i)
    elif prediction == 'Shirt':
        shirts.append(i)
    else:
        unidentified.append(i)


output = f"Dresses: {len(dress)} \nPants: {len(pants)} \nShirts: {len(shirts)} \nShoes: {len(shoes)} \nUnidentified: {len(unidentified)}"
print(f"Inventory Summary: \n{output}")

print("\nItem categories and file name")
print(f"Dresses: {dress} \n\nPants: {pants} \n\nShirts: {shirts} \n\nShoes: {shoes}")
if len(unidentified) > 0:
    print(f"Unidentifed: {unidentified}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━